In [1]:
###########################
### Author: Hakan Toguc ###
### Date : 17-Jun-2021  ###
###########################

# This study intended to forecast bus demands of municipalities in Banana Republic

import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from sklearn.metrics import mean_absolute_error
from collections import OrderedDict

In [2]:
# Reading the dataset file and performing operations on it 
df_raw = pd.read_csv("municipality_bus_utilization.csv", header=0,)
df_raw.sort_values(["timestamp",],inplace=True,ascending=True)
df_raw.reset_index(drop=True,inplace=True)

# In order to scale "usage" parameter we used "usage/total_capacity"
df_raw["usage_ratio"] = df_raw["usage"] / df_raw["total_capacity"]

# we record each municipality's total_capacity into a dictionary variable
dict_capacity = dict()
for row in df_raw.itertuples():
    if row.municipality_id not in dict_capacity:
        dict_capacity[row.municipality_id] = row.total_capacity
df_raw.drop(columns=["usage","total_capacity"],inplace=True)
df_raw['timestamp_ts'] = pd.to_datetime(df_raw['timestamp'])

# In order to define start and end of one week interval, we find out day of week
df_raw["dayofweek"] = df_raw['timestamp_ts'].dt.dayofweek
df_raw["date"] = df_raw["timestamp"].apply(func=lambda x: str(x)[:10])

# if minute value of timestamp is above 45 then we accept it as next hour value
df_raw["order"] = df_raw["timestamp"].apply(
    func=lambda x: int(str(x)[11:13]) + 1 if str(x)[14:16] > "45" else int(str(x)[11:13]))

# we accept largest usage_ratio out of more than one usage data during an hour
df_hourly = df_raw.groupby(["date", "municipality_id", "dayofweek",
                "order"]).agg({'usage_ratio': 'max', })[['usage_ratio', ]].reset_index()
df_hourly = df_hourly.sort_values(["date", "order"])

# pivot table is generated to sum up daily data on a single line
df_pivot = pd.pivot_table(data=df_hourly, values="usage_ratio",
                               index=["date", "municipality_id", "dayofweek"], columns="order")
df_pivot.reset_index(drop=False, inplace=True)

# test part of the data is seperated
df_testweek_1 = df_pivot.loc[(df_pivot["date"] > "2017-08-05") &
                                       (df_pivot["date"] < "2017-08-13")].reset_index(drop=True)
df_testweek_2 = df_pivot.loc[(df_pivot["date"] > "2017-08-12")].reset_index(drop=True)
df_pivot.drop(df_pivot[df_pivot["date"] > "2017-08-05"].index, inplace=True)

# missing dates' values are calculated using median, depending on which day of the week
weekday_medians = df_pivot.groupby(by=["dayofweek", "municipality_id"], ).median()

# missing dates and their dayofweek values are given below
missing_days = [("2017-06-20", 1), ("2017-06-21", 2), ("2017-07-31", 0), ("2017-08-03", 3), ("2017-08-04", 4)]
municipality_count = df_raw["municipality_id"].nunique()
for day in missing_days:
    for municipality_id in range(municipality_count):
        # missing rows are gerenerated
        new_row = {"date": day[0], "municipality_id": municipality_id, "dayofweek": day[1]}
        for hour in range(8, 17):
            new_row[hour] = weekday_medians.loc[(day[1], municipality_id), hour]
        # newly generated missing row is appended to training data
        df_pivot = df_pivot.append(new_row, ignore_index=True)
df_pivot = df_pivot.sort_values("date").reset_index(drop=True)


In [3]:
# our training data with usage ratio given in hour columns (8,9,10,.....16) 
print(df_pivot.head())

         date  municipality_id  dayofweek         8         9        10  \
0  2017-06-04                0          6  0.087807  0.152506  0.189833   
1  2017-06-04                1          6  0.347607  0.536524  0.687657   
2  2017-06-04                2          6  0.446198  0.606887  0.773314   
3  2017-06-04                3          6  0.398964  0.585492  0.690674   
4  2017-06-04                4          6  0.379142  0.645004  0.781916   

         11        12        13        14        15        16  
0  0.212229  0.218272  0.215073  0.207963  0.191255  0.167081  
1  0.838791  0.863980  0.861461  0.858942  0.795970  0.702771  
2  0.879484  0.942611  0.946915  0.926829  0.863702  0.746055  
3  0.744560  0.761140  0.754922  0.745596  0.720207  0.635751  
4  0.829694  0.837400  0.832520  0.810172  0.775751  0.684562  


In [4]:
# First test week data
print(df_testweek_1.head())

order        date  municipality_id  dayofweek         8         9        10  \
0      2017-08-06                0          6  0.116957  0.163171  0.202275   
1      2017-08-06                1          6  0.544081  0.662469  0.838791   
2      2017-08-06                2          6  0.446198  0.558106  0.720230   
3      2017-08-06                3          6  0.406218  0.595337  0.692746   
4      2017-08-06                4          6  0.549705  0.723863  0.810172   

order        11        12        13        14        15        16  
0      0.230359  0.232137  0.233914  0.229293  0.212584  0.188411  
1      1.002519  1.000000  1.005038  1.007557  0.942065  0.863980  
2      0.807747  0.862267  0.939742  0.932568  0.859397  0.784792  
3      0.792228  0.805181  0.807254  0.807254  0.755959  0.665803  
4      0.875417  0.884151  0.883380  0.876702  0.827126  0.729258  


In [5]:
# Second test week data
print(df_testweek_2.head())

order        date  municipality_id  dayofweek         8         9        10  \
0      2017-08-13                0          6  0.111625  0.162105  0.195876   
1      2017-08-13                1          6  0.549118  0.710327  0.889169   
2      2017-08-13                2          6  0.690100  0.822095  0.880918   
3      2017-08-13                3          6  0.399482  0.582383  0.719689   
4      2017-08-13                4          6  0.438736  0.668636  0.822759   

order        11        12        13        14        15        16  
0      0.219339  0.225738  0.226093  0.225027  0.199431  0.175613  
1      1.010076  1.032746  1.035264  1.025189  0.924433  0.851385  
2      0.807747  0.889527  0.928264  0.959828  0.855093  0.737446  
3      0.779793  0.805181  0.810363  0.800000  0.756477  0.656477  
4      0.886720  0.901105  0.898793  0.886720  0.846134  0.745441  


In [6]:
# Each municipality handled alone to generate ML models, thus there are 10 models in total
# Evaluations are performed after combining the predictions of all models 

In [7]:
# to store municipality based evaluations
evaluation_dict = OrderedDict()
y_true_week1_total = []
y_pred_week1_total = []
y_true_week2_total = []
y_pred_week2_total = []

In [8]:
def train(municipality_id):
    """ Model design, data customization and training are performed here
        only one municipality's model is trained at ones """
    train_data = df_pivot.loc[df_pivot["municipality_id"] == municipality_id]
    train_data = train_data.sort_values(by=["date",]).reset_index(drop=True)
    # hourly usage_ratio values are used as training data
    train_data = train_data[[8,9,10,11,12,13,14,15,16]].values
    X = []
    y = []
    # we used 7-day-data as one sequence and customized input and labeled data accordingly
    loop_count = int(len(train_data)/7)
    for ind in range(loop_count):
        if ind == loop_count-1:
            break
        X.append(train_data[ind*7:(ind+1)*7])
        y.append(train_data[(ind+1)*7:(ind+2)*7])
    X = np.array(X)
    y = np.array(y)

    # setting up our neural network model
    # input shape is 7-day-data of one municipality
    # for each day there are 9 hours usage data
    model = Sequential()
    model.add(LSTM(126,input_shape=(7,9),))
    model.add(RepeatVector(7))
    model.add(LSTM(126, return_sequences=True))
    model.add(Dense(126,))
    model.add(Dense(63,))
    model.add(TimeDistributed(Dense(9,)))
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae'])
    model.fit(X, y, epochs=60, batch_size=5, verbose=1)
    return model,y

In [9]:
def evaluation_test_week_1(municipality_id, model, y):
    """ First test week evaluation of each municipality is performed """
    global evaluation_dict,y_true_week1_total,y_pred_week1_total
    # to predict the first test week, we use last week of training data as input
    y_pred_week1 = model.predict(y[-1].reshape(1, y[-1].shape[0], y[-1].shape[1]))
    # we filter single municipality's test week 1 data
    df_testweek1_filt = df_testweek_1.loc[df_testweek_1["municipality_id"] == municipality_id]
    df_testweek1_filt = df_testweek1_filt.sort_values(["date",]).reset_index(drop=True)
    np_testweek1_filt = df_testweek1_filt[[8, 9, 10, 11, 12, 13, 14, 15, 16]].values
    # we transform one week 2-dim array datas to flat arrays, to facilitate evaluation
    y_pred_week1_flat = np.reshape(y_pred_week1,(-1,))
    np_testweek1_filt_flat = np.reshape(np_testweek1_filt,(-1,))
    # we used real usage values to calculate error metric, not usage_ratios like in training
    mae1 = mean_absolute_error(y_true=np_testweek1_filt_flat * dict_capacity[municipality_id],
                                      y_pred=y_pred_week1_flat * dict_capacity[municipality_id])
    print("mae_week1 for municipality = {} is {:.2f}".format(municipality_id, mae1))
    evaluation_dict[municipality_id] = [mae1,]
    y_true_week1_total = y_true_week1_total + list(np_testweek1_filt_flat * dict_capacity[municipality_id])
    y_pred_week1_total += list(y_pred_week1_flat * dict_capacity[municipality_id])
    return model,np_testweek1_filt


In [10]:
def evaluation_test_week_2(municipality_id, model, np_testweek1_filt):
    """ Second test week evaluation of each municipality is performed """
    global evaluation_dict,y_true_week2_total,y_pred_week2_total
    # to predict the second test week, we use first test week data as input
    y_pred_week2 = model.predict(np_testweek1_filt.reshape(1, np_testweek1_filt.shape[0],
                                                                np_testweek1_filt.shape[1]))
    # we filter single municipality's test week 2 data
    df_testweek2_filt = df_testweek_2.loc[df_testweek_2["municipality_id"] == municipality_id]
    df_testweek2_filt = df_testweek2_filt.sort_values(["date", ]).reset_index(drop=True)
    np_testweek2_filt = df_testweek2_filt[[8, 9, 10, 11, 12, 13, 14, 15, 16]].values
    # we transform one week 2-dim array datas to flat arrays, to facilitate evaluation
    y_pred_week2_flat = np.reshape(y_pred_week2,(-1,))
    np_testweek2_filt_flat = np.reshape(np_testweek2_filt,(-1,))
    # we used real usage values to calculate error metric, not usage_ratios like in training
    mae2 = mean_absolute_error(y_true=np_testweek2_filt_flat * dict_capacity[municipality_id],
                                      y_pred=y_pred_week2_flat * dict_capacity[municipality_id])
    print("mae_week2 for municipality = {} is {:.2f}".format(municipality_id, mae2))
    evaluation_dict[municipality_id].append(mae2)
    y_true_week2_total += list(np_testweek2_filt_flat * dict_capacity[municipality_id])
    y_pred_week2_total += list(y_pred_week2_flat * dict_capacity[municipality_id])

In [11]:
def overall_evaluation():
    """ Single test week and both test weeks evaluation of all municipalities are performed """
    # First test week error calculation for all municipalities combined
    mae_week_1 = mean_absolute_error(y_true=y_true_week1_total,y_pred=y_pred_week1_total)
    print()
    print("**************** RESULTS *********************")
    print()
    print("mae_week1 for all municipalities is {:.2f}".format(mae_week_1))
    # Second test week error calculation for all municipalities combined
    mae_week_2 = mean_absolute_error(y_true=y_true_week2_total, y_pred=y_pred_week2_total)
    print("mae_week2 for all municipalities is {:.2f}".format(mae_week_2))
    # Ultimate error calculation of both test weeks and all municipalities combined
    mae_overall = mean_absolute_error(y_true=y_true_week1_total+y_true_week2_total,
                                      y_pred=y_pred_week1_total+y_pred_week2_total)
    print("mae_overall for all municipalities and both weeks combined is {:.2f}".format(mae_overall))

In [13]:
municipality_count = 10
for municipality_id in range(municipality_count):
    model,y = train(municipality_id)
    model,np_testweek1_filt = evaluation_test_week_1(municipality_id, model, y)
    evaluation_test_week_2(municipality_id, model, np_testweek1_filt)
print()
print("Dictionary of each municipality MAE evaluation on TestWeek 1 and TestWeek 2")
print("---------------------------------------------------------------------------")
print(evaluation_dict)
overall_evaluation()

# **************** RESULTS *********************
#
# mae_week1 for all municipalities is 118.58
# mae_week2 for all municipalities is 161.24
# mae_overall for all municipalities and both weeks combined is 139.91

Epoch 1/60
2/2 [==============================] - 3s 11ms/step - loss: 0.1866 - mae: 0.1866
Epoch 2/60
2/2 [==============================] - 0s 12ms/step - loss: 0.1074 - mae: 0.1074
Epoch 3/60
2/2 [==============================] - 0s 12ms/step - loss: 0.0884 - mae: 0.0884
Epoch 4/60
2/2 [==============================] - 0s 13ms/step - loss: 0.0839 - mae: 0.0839
Epoch 5/60
2/2 [==============================] - 0s 13ms/step - loss: 0.0728 - mae: 0.0728
Epoch 6/60
2/2 [==============================] - 0s 13ms/step - loss: 0.0692 - mae: 0.0692
Epoch 7/60
2/2 [==============================] - 0s 11ms/step - loss: 0.0607 - mae: 0.0607
Epoch 8/60
2/2 [==============================] - 0s 10ms/step - loss: 0.0560 - mae: 0.0560
Epoch 9/60
2/2 [==============================] - 0s 10ms/step - loss: 0.0538 - mae: 0.0538
Epoch 10/60
2/2 [==============================] - 0s 10ms/step - loss: 0.0516 - mae: 0.0516
Epoch 11/60
2/2 [==============================] - 0s 11ms/step - loss: 0.0468 

2/2 [==============================] - 0s 9ms/step - loss: 0.0895 - mae: 0.0895
Epoch 29/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0835 - mae: 0.0835
Epoch 30/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0826 - mae: 0.0826
Epoch 31/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0830 - mae: 0.0830
Epoch 32/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0794 - mae: 0.0794
Epoch 33/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0825 - mae: 0.0825
Epoch 34/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0808 - mae: 0.0808
Epoch 35/60
2/2 [==============================] - 0s 8ms/step - loss: 0.0775 - mae: 0.0775
Epoch 36/60
2/2 [==============================] - 0s 12ms/step - loss: 0.0817 - mae: 0.0817
Epoch 37/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0791 - mae: 0.0791
Epoch 38/60
2/2 [==============================] - 0s 10ms/step - loss: 0.0770 - mae: 0.077

2/2 [==============================] - 0s 8ms/step - loss: 0.0787 - mae: 0.0787
Epoch 57/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0806 - mae: 0.0806
Epoch 58/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0769 - mae: 0.0769
Epoch 59/60
2/2 [==============================] - 0s 8ms/step - loss: 0.0798 - mae: 0.0798
Epoch 60/60
2/2 [==============================] - 0s 8ms/step - loss: 0.0775 - mae: 0.0775
mae_week1 for municipality = 2 is 55.00
mae_week2 for municipality = 2 is 56.49
Epoch 1/60
2/2 [==============================] - 3s 13ms/step - loss: 0.5674 - mae: 0.5674
Epoch 2/60
2/2 [==============================] - 0s 11ms/step - loss: 0.3910 - mae: 0.3910
Epoch 3/60
2/2 [==============================] - 0s 11ms/step - loss: 0.2739 - mae: 0.2739
Epoch 4/60
2/2 [==============================] - 0s 11ms/step - loss: 0.2507 - mae: 0.2507
Epoch 5/60
2/2 [==============================] - 0s 11ms/step - loss: 0.2209 - mae: 0.2209
Epoch 6/60
2

2/2 [==============================] - 0s 10ms/step - loss: 0.1227 - mae: 0.1227
Epoch 23/60
2/2 [==============================] - 0s 10ms/step - loss: 0.1219 - mae: 0.1219
Epoch 24/60
2/2 [==============================] - 0s 10ms/step - loss: 0.1212 - mae: 0.1212
Epoch 25/60
2/2 [==============================] - 0s 9ms/step - loss: 0.1213 - mae: 0.1213
Epoch 26/60
2/2 [==============================] - 0s 10ms/step - loss: 0.1218 - mae: 0.1218
Epoch 27/60
2/2 [==============================] - 0s 10ms/step - loss: 0.1237 - mae: 0.1237
Epoch 28/60
2/2 [==============================] - 0s 10ms/step - loss: 0.1227 - mae: 0.1227
Epoch 29/60
2/2 [==============================] - 0s 10ms/step - loss: 0.1178 - mae: 0.1178
Epoch 30/60
2/2 [==============================] - 0s 11ms/step - loss: 0.1170 - mae: 0.1170
Epoch 31/60
2/2 [==============================] - 0s 12ms/step - loss: 0.1225 - mae: 0.1225
Epoch 32/60
2/2 [==============================] - 0s 11ms/step - loss: 0.1180 - ma

2/2 [==============================] - 0s 9ms/step - loss: 0.0728 - mae: 0.0728
Epoch 50/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0720 - mae: 0.0720
Epoch 51/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0720 - mae: 0.0720
Epoch 52/60
2/2 [==============================] - 0s 8ms/step - loss: 0.0726 - mae: 0.0726
Epoch 53/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0714 - mae: 0.0714
Epoch 54/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0706 - mae: 0.0706
Epoch 55/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0711 - mae: 0.0711
Epoch 56/60
2/2 [==============================] - 0s 10ms/step - loss: 0.0704 - mae: 0.0704
Epoch 57/60
2/2 [==============================] - 0s 10ms/step - loss: 0.0708 - mae: 0.0708
Epoch 58/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0708 - mae: 0.0708
Epoch 59/60
2/2 [==============================] - 0s 8ms/step - loss: 0.0704 - mae: 0.070

2/2 [==============================] - 0s 15ms/step - loss: 0.0810 - mae: 0.0810
Epoch 16/60
2/2 [==============================] - 0s 14ms/step - loss: 0.0797 - mae: 0.0797
Epoch 17/60
2/2 [==============================] - 0s 15ms/step - loss: 0.0735 - mae: 0.0735
Epoch 18/60
2/2 [==============================] - 0s 16ms/step - loss: 0.0740 - mae: 0.0740
Epoch 19/60
2/2 [==============================] - 0s 11ms/step - loss: 0.0691 - mae: 0.0691
Epoch 20/60
2/2 [==============================] - 0s 11ms/step - loss: 0.0664 - mae: 0.0664
Epoch 21/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0625 - mae: 0.0625
Epoch 22/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0611 - mae: 0.0611
Epoch 23/60
2/2 [==============================] - 0s 10ms/step - loss: 0.0598 - mae: 0.0598
Epoch 24/60
2/2 [==============================] - 0s 11ms/step - loss: 0.0563 - mae: 0.0563
Epoch 25/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0546 - mae:

2/2 [==============================] - 0s 10ms/step - loss: 0.0409 - mae: 0.0409
Epoch 43/60
2/2 [==============================] - 0s 13ms/step - loss: 0.0395 - mae: 0.0395
Epoch 44/60
2/2 [==============================] - 0s 14ms/step - loss: 0.0397 - mae: 0.0397
Epoch 45/60
2/2 [==============================] - 0s 13ms/step - loss: 0.0395 - mae: 0.0395
Epoch 46/60
2/2 [==============================] - 0s 13ms/step - loss: 0.0385 - mae: 0.0385
Epoch 47/60
2/2 [==============================] - 0s 13ms/step - loss: 0.0384 - mae: 0.0384
Epoch 48/60
2/2 [==============================] - 0s 13ms/step - loss: 0.0386 - mae: 0.0386
Epoch 49/60
2/2 [==============================] - 0s 13ms/step - loss: 0.0383 - mae: 0.0383
Epoch 50/60
2/2 [==============================] - 0s 13ms/step - loss: 0.0383 - mae: 0.0383
Epoch 51/60
2/2 [==============================] - 0s 10ms/step - loss: 0.0377 - mae: 0.0377
Epoch 52/60
2/2 [==============================] - 0s 9ms/step - loss: 0.0370 - ma